<a href="https://colab.research.google.com/github/s2t2/openai-embeddings-2023/blob/main/notebooks/1-sampling/Sample_of_Users_by_Account_Type_with_Tweet_Minimum_20230702.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's pull a balanced random sample of users representing each account type (by opinion, bot status, qanon status), as well as their tweets (max 25 or 50 should be plenty).

We concat the tweet text with a double pipe, like "tweet 1 text || tweet 2 text || tweet 3 text || etc." for human readability.


## Setup

### Mounting the Drive

In [ ]:
#import os
#
#from google.colab import drive
#
#drive.mount('/content/drive')
#print(os.listdir(os.getcwd()))

In [ ]:
## you might need to create a google drive SHORTCUT that has this same path
## ... or update the path to use your own google drive organization
#DIRPATH = '/content/drive/MyDrive/Research/DS Research Shared 2023'
#print(DIRPATH)
#os.path.isdir(DIRPATH)

### BigQuery Service

In [1]:
from google.colab import auth

# asks you to login
auth.authenticate_user()

In [8]:

from google.cloud import bigquery
from pandas import DataFrame

PROJECT_ID = "tweet-research-shared"

class BigQueryService():
    def __init__(self):
        self.client = bigquery.Client(project=PROJECT_ID)

    def execute_query(self, sql, verbose=True):
        if verbose == True:
            print(sql)
        job = self.client.query(sql)
        return job.result()

    def query_to_df(self, sql, verbose=True):
        """high-level wrapper to return a DataFrame"""
        results = self.execute_query(sql, verbose=verbose)
        records = [dict(row) for row in list(results)]
        df = DataFrame(records)
        return df


In [15]:
bq = BigQueryService()
print("PROJECT:", bq.client.project)

PROJECT: tweet-research-shared


## Usage

### Users Sample

In [22]:
USERS_PER_GROUP = int(3000 / 6)
print(USERS_PER_GROUP)

500


In [62]:
GROUPS = [
    # anti-trump
    {"opinion_community": 0, "is_bot": "false", "is_q": "false"},
    {"opinion_community": 0, "is_bot": "true",  "is_q": "false"},
    # pro-trump
    {"opinion_community": 1, "is_bot": "false", "is_q": "false"},
    {"opinion_community": 1, "is_bot": "true",  "is_q": "false"},
    # q-anon
    {"opinion_community": 1, "is_bot": "false", "is_q": "true"},
    {"opinion_community": 1, "is_bot": "true",  "is_q": "true"},
]


#### No Tweet Minimum

In [63]:
from pandas import concat

frames = []
for group in GROUPS:
    opinion_community = group["opinion_community"] #> 0
    is_bot = group["is_bot"] #> "false"
    is_q = group["is_q"] #> "false"
    print("USER GROUP:...", opinion_community, "| BOT:", is_bot, "| Q:",is_q)

    sql = f"""
        SELECT
            u.user_id
            ,u.opinion_community
            ,u.is_bot
            ,u.is_q
            ,u.created_on
            ,u.screen_name_count
            ,u.status_count
            ,u.rt_count
        FROM `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u
        WHERE u.opinion_community={opinion_community} and u.is_bot={is_bot} and u.is_q={is_q}

        ORDER BY rand()
        LIMIT {int(USERS_PER_GROUP)};
    """
    df = bq.query_to_df(sql, verbose=False)
    #print(df.head())
    frames.append(df)

users_df = concat(frames)
print(len(users_df))
users_df.to_csv(f"users_sample_by_account_type_20230702_{len(users_df)}.csv")
users_df

USER GROUP:... 0 | BOT: false | Q: false
USER GROUP:... 0 | BOT: true | Q: false
USER GROUP:... 1 | BOT: false | Q: false
USER GROUP:... 1 | BOT: true | Q: false
USER GROUP:... 1 | BOT: false | Q: true
USER GROUP:... 1 | BOT: true | Q: true
3000


,user_id,opinion_community,is_bot,is_q,created_on,screen_name_count,status_count,rt_count
0,991576458001879040,0,False,False,2018-05-02,1,1,0
1,885056257865986048,0,False,False,2017-07-12,1,1,1
2,909549194314080256,0,False,False,2017-09-17,1,1,0
3,2849515431,0,False,False,2014-10-29,1,1,1
4,32895294,0,False,False,2009-04-18,1,45,28
...,...,...,...,...,...,...,...,...
495,4029018734,1,True,True,2015-10-26,1,1768,1249
496,832715691467931648,1,True,True,2017-02-17,1,519,514
497,904533645859414016,1,True,True,2017-09-04,1,943,929
498,69436228,1,True,True,2009-08-27,1,1029,878


In [64]:
users_df[["opinion_community", "is_q", "is_bot"]].value_counts()

opinion_community  is_q   is_bot
0                  False  False     500
                          True      500
1                  False  False     500
                          True      500
                   True   False     500
                          True      500
dtype: int64

Let's try to get a sense of the distribution of number of tweets for each group.

In [65]:
users_df.groupby(["opinion_community", "is_q", "is_bot"])["status_count"].describe()

count     mean         std   min    25%  \
opinion_community is_q  is_bot                                            
0                 False False   500.0    9.380   39.468356   1.0    1.0   
                        True    500.0  863.264  624.418619  86.0  507.5   
1                 False False   500.0   17.342   59.935244   1.0    1.0   
                        True    500.0  823.620  569.194350  30.0  461.0   
                  True  False   500.0  121.186  143.710559   1.0   19.0   
                        True    500.0  742.840  382.601047  97.0  468.0   

                                  50%      75%     max  
opinion_community is_q  is_bot                          
0                 False False     1.0     4.25   725.0  
                        True    754.0  1069.25  7732.0  
1                 False False     2.0     6.00   548.0  
                        True    690.0  1053.50  4678.0  
                  True  False    63.0   179.25   878.0  
                        True    703.0   943.25  2649.0

We see some groups like non-Q humans have low average and median number of tweets.

We could consider re-pulling the sample, but this time including only users who have a higher number of tweets than some minimum threshold. If the threshold is too low, we might not have enough signal. If the threshold is too high, we risk biasing the sample. Need to think more about this.

In [69]:

tweet_counts_df = users_df.groupby(["opinion_community", "is_q", "is_bot"])["status_count"]
tweet_counts_df.quantile(.7)
tweet_counts_df.quantile(.75)
tweet_counts_df.quantile(.8)
tweet_counts_df.quantile(.85)

opinion_community  is_q   is_bot
0                  False  False       10.00
                          True      1292.05
1                  False  False       14.15
                          True      1281.40
                   True   False      258.60
                          True      1078.45
Name: status_count, dtype: float64

In [78]:
humans_df = users_df.copy()
humans_df = humans_df[ humans_df["is_bot"]==False]
humans_df = humans_df[ humans_df["is_q"]==False ]
humans_df.groupby("opinion_community")["status_count"].describe()

,count,mean,std,min,25%,50%,75%,max
opinion_community,,,,,,,,
0,500.0,9.380,39.468356,1.0,1.0,1.0,4.25,725.0
1,500.0,17.342,59.935244,1.0,1.0,2.0,6.00,548.0


In [81]:
import plotly.express as px

px.violin(humans_df, x="status_count", points="all", box=True)

In [84]:
px.histogram(humans_df, x="status_count", log_y=True)

In [121]:
import numpy as np

#percentiles = np.linspace(start=0.5, stop=0.95, num=20)
#percentiles = [x / 1000 for x in range(550, 950, 25)]
percentiles = [0.55, .6, .65, .7, .75, .8, .825, .83, .85, .875, .9, .95]
for pctile in percentiles:
    print(pctile, "...", humans_df["status_count"].quantile(pctile))


0.55 ... 2.0
0.6 ... 2.0
0.65 ... 3.0
0.7 ... 4.0
0.75 ... 5.0
0.8 ... 8.0
0.825 ... 9.0
0.83 ... 10.0
0.85 ... 12.0
0.875 ... 16.0
0.9 ... 18.0
0.95 ... 52.09999999999991


Looks like we could maybe choose 8-10 minimum, in which case we would be getting around the top ~20% of most frequent tweeters.

#### 10 Tweet Minimum

In [124]:
from pandas import concat

# controls which users to include in the sample:
TWEET_MIN = 10

frames = []
for group in GROUPS:
    opinion_community = group["opinion_community"] #> 0
    is_bot = group["is_bot"] #> "false"
    is_q = group["is_q"] #> "false"
    print("USER GROUP:...", opinion_community, "| BOT:", is_bot, "| Q:",is_q)

    sql = f"""
        SELECT
            u.user_id
            ,u.opinion_community
            ,u.is_bot
            ,u.is_q
            ,u.created_on
            ,u.screen_name_count
            ,u.status_count
            ,u.rt_count
        FROM `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u
        WHERE u.opinion_community={opinion_community} and u.is_bot={is_bot} and u.is_q={is_q}
            AND u.status_count >= {int(TWEET_MIN)}
        ORDER BY rand()
        LIMIT {int(USERS_PER_GROUP)};
    """
    df = bq.query_to_df(sql, verbose=False)
    #print(df.head())
    frames.append(df)

users_df = concat(frames)
print(len(users_df))
users_df.to_csv(f"users_sample_by_account_type_20230702_{len(users_df)}_min_{TWEET_MIN}.csv")
users_df

USER GROUP:... 0 | BOT: false | Q: false
USER GROUP:... 0 | BOT: true | Q: false
USER GROUP:... 1 | BOT: false | Q: false
USER GROUP:... 1 | BOT: true | Q: false
USER GROUP:... 1 | BOT: false | Q: true
USER GROUP:... 1 | BOT: true | Q: true
3000


,user_id,opinion_community,is_bot,is_q,created_on,screen_name_count,status_count,rt_count
0,824346638391136271,0,False,False,2017-01-25,1,25,5
1,755966161,0,False,False,2012-08-13,1,70,22
2,761045801200029696,0,False,False,2016-08-04,1,220,217
3,965738511230844928,0,False,False,2018-02-20,1,196,61
4,492802179,0,False,False,2012-02-15,1,10,9
...,...,...,...,...,...,...,...,...
495,1005860415044227072,1,True,True,2018-06-10,1,595,595
496,331456884,1,True,True,2011-07-08,1,906,877
497,4206962773,1,True,True,2015-11-12,1,611,572
498,804332988171505664,1,True,True,2016-12-01,1,1146,1146


In [125]:
users_df.groupby(["opinion_community", "is_q", "is_bot"])["status_count"].describe()

count     mean         std   min    25%  \
opinion_community is_q  is_bot                                            
0                 False False   500.0   71.310  133.014027  10.0   15.0   
                        True    500.0  950.168  604.088020  90.0  561.5   
1                 False False   500.0   72.438  106.629611  10.0   15.0   
                        True    500.0  818.688  560.130452  34.0  436.5   
                  True  False   500.0  152.126  162.881512  10.0   37.0   
                        True    500.0  758.514  390.711254  54.0  494.5   

                                  50%      75%     max  
opinion_community is_q  is_bot                          
0                 False False    27.0    64.00  1725.0  
                        True    807.5  1195.50  3570.0  
1                 False False    30.0    72.50   713.0  
                        True    676.0  1039.50  4056.0  
                  True  False    94.0   212.25  1317.0  
                        True    697.5   954.75  2367.0

Now the median of each group looks to be above 25 tweets, so we can take a maximum of around 25 or 30 of their tweets and have roughly the same amount of signal for each group.

In [126]:
users_df.groupby(["opinion_community", "is_q", "is_bot"])["status_count"].median()

opinion_community  is_q   is_bot
0                  False  False      27.0
                          True      807.5
1                  False  False      30.0
                          True      676.0
                   True   False      94.0
                          True      697.5
Name: status_count, dtype: float64

In [128]:
humans_df = users_df.copy()
humans_df = humans_df[ humans_df["is_bot"]==False]
humans_df = humans_df[ humans_df["is_q"]==False ]
humans_df.groupby("opinion_community")["status_count"].describe()

,count,mean,std,min,25%,50%,75%,max
opinion_community,,,,,,,,
0,500.0,71.310,133.014027,10.0,15.0,27.0,64.0,1725.0
1,500.0,72.438,106.629611,10.0,15.0,30.0,72.5,713.0


In [144]:
px.histogram(humans_df, x="status_count", log_y=False)

In [146]:
import numpy as np

#percentiles = np.linspace(start=0.5, stop=0.95, num=20)
#percentiles = [x / 1000 for x in range(550, 950, 25)]
percentiles = [0.3, 0.35, 0.4, 0.45, 0.5, 0.55, .6, .65, .7, .75, .8, .85, .9, .95]
for pctile in percentiles:
    print(pctile, "...", humans_df["status_count"].quantile(pctile))


0.3 ... 17.0
0.35 ... 19.0
0.4 ... 21.0
0.45 ... 24.0
0.5 ... 28.0
0.55 ... 32.0
0.6 ... 37.0
0.65 ... 45.35000000000002
0.7 ... 55.0
0.75 ... 69.0
0.8 ... 95.20000000000005
0.85 ... 130.14999999999998
0.9 ... 188.20000000000005
0.95 ... 293.0999999999999


### Tweets from those Users

In [127]:
user_ids = users_df["user_id"].tolist()
print(len(user_ids))
print(len(list(set(user_ids))))
print(user_ids[0:10])

3000
3000
[824346638391136271, 755966161, 761045801200029696, 965738511230844928, 492802179, 129642515, 1223394235593846784, 63337810, 10717612, 725846092340977664]


In [147]:
from pandas import concat

# controls how much max signal from the users we have sampled:
TWEET_MAX = 30

frames = []
for user_id in user_ids:

    sql = f"""
        SELECT
            u.user_id
            ,u.created_on
            ,u.screen_name_count
            ,u.screen_names
            ,u.status_count
            ,u.rt_count
            ,(u.rt_count / u.status_count) as rt_pct

            ,u.opinion_community
            ,u.is_bot
            ,u.is_q

            ,up.descriptions as profile_descriptions

            ,string_agg(t.status_text, ' || ' ORDER BY rand() LIMIT {TWEET_MAX}) as tweet_texts

        FROM `tweet-research-shared.impeachment_2020.tweets_v2` t
        JOIN `tweet-research-shared.impeachment_2020.user_details_v20210806_slim` u ON u.user_id = t.user_id
        LEFT JOIN `tweet-research-shared.impeachment_2020.user_profiles_v2` up on up.user_id = u.user_id
        WHERE t.user_id = {user_id}
        GROUP BY 1,2,3,4,5,6,7,8,9,10,11
    """
    df = bq.query_to_df(sql, verbose=False)
    #print(df.head())
    frames.append(df)
    print(len(frames))

user_tweets_df = concat(frames)
print(len(user_tweets_df))
user_tweets_df.to_csv(f"users_sample_by_account_type_20230702_{len(users_df)}_min_{TWEET_MIN}_max_{TWEET_MAX}_tweets.csv")
user_tweets_df

3000


,user_id,created_on,screen_name_count,screen_names,status_count,rt_count,rt_pct,opinion_community,is_bot,is_q,profile_descriptions,tweet_texts
0,824346638391136271,2017-01-25,1,ROGUEBARKEEP,25,5,0.200000,0,False,False,TRAINER OF CIRCUS SHRIMP - LOVER OF @PIZZAPUFF...,@realDonaldTrump THIS IS WHY WE NEEDED IMPEACH...
0,755966161,2012-08-13,1,POPPYPHIL1,70,22,0.314286,0,False,False,"RETIRED PHYSICIAN, COMMITTED LIFELONG PROGRESS...",RT @SethAbramson: BREAKING NEWS: All Four Roge...
0,761045801200029696,2016-08-04,1,VEE_JORDAN1,220,217,0.986364,0,False,False,,RT @RepValDemings: When we impeached the presi...
0,965738511230844928,2018-02-20,1,MICHELIAFIDO,196,61,0.311224,0,False,False,"""WHO THE HELL CAN SEE FOREVER?""","That is some serious, NoKo-level propaganda. |..."
0,492802179,2012-02-15,1,OUROBOROSLEGION,10,9,0.900000,0,False,False,JUST A GUY. BASEBALL AND VIDEO GAMES ARE MY TH...,RT @BarbMcQuade: Here are the GOP defenses I h...
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1005860415044227072,2018-06-10,1,JUSTABITBAD1,595,595,1.000000,1,True,True,"🇺🇸🇺🇸🇺🇸🇺🇸🇺🇸 SON, FATHER, BUSINESS OWNER, CONSER...",RT @KaraMar111: @AOC Idiots like U beg IlLegal...
0,331456884,2011-07-08,1,TARAENDTHEFED,906,877,0.967991,1,True,True,"HI, I'M WORKING FOR A FULL AND COMPLETE AUDIT ...",RT @Cernovich: NEARLY 47 MILLION VIEWERS WATCH...
0,4206962773,2015-11-12,1,LJJWYO,611,572,0.936170,1,True,True,"PRO PRIVATE HEALTHCARE OPTIONS, PATRIOTISM, LO...",RT @JonathanTurley: Rather than wait for court...
0,804332988171505664,2016-12-01,1,VENOM_KILLER21,1146,1146,1.000000,1,True,True,,RT @OliverMcGee: Blacks are voting @realDonald...
